In [1]:
import numpy as np

In [2]:
def FRF_matrix(m, k, alpha, beta, f_start=0.01, f_end=5, f_resolution=0.01):
    """
    Calculate structure FRFs for a free–free chain of masses & springs
    with Rayleigh damping C = alpha*M + beta*K.

    Parameters
    ----------
    m : array_like, shape (n,)
        Masses at the n DOFs.
    k : array_like, shape (n-1,)
        Stiffnesses of the springs between DOFs.
    alpha, beta : float
        Rayleigh damping coefficients.
    f_start, f_end, f_resolution : floats
        Frequency sweep (Hz).

    Returns
    -------
    freq : ndarray, shape (n_freq,)
        Frequencies in Hz.
    Y    : ndarray, shape (n, n, n_freq), complex
        FRF tensor, Y[:, :, i] = inv[ K + i ω C − ω² M ] at freq[i].
    """
    # frequency vector
    freq  = np.arange(f_start, f_end, f_resolution)
    omega = 2 * np.pi * freq          # rad/s
    n     = len(m)

    # Mass matrix
    M = np.diag(m)

    # Stiffness matrix K for free–free chain
    # main diagonal: [k[0], k[0]+k[1], ..., k[n-2]+k[n-1], k[n-1]]
    main_diag = np.empty(n)
    main_diag[0]   = k[0]
    main_diag[-1]  = k[-1]
    main_diag[1:-1] = k[:-1] + k[1:]
    # off-diagonals: -k between adjacent DOFs
    K = np.diag(main_diag) \
      + np.diag(-k, 1)   \
      + np.diag(-k, -1)

    # Damping matrix
    C = alpha*M + beta*K

    # Allocate FRF
    n_freq = len(freq)
    Y = np.empty((n_freq,n,n), dtype=complex)

    # Loop over frequencies, build & invert each D(ω)
    for i, ω in enumerate(omega):
        D = K + 1j*ω*C - (ω**2)*M
        Y[i,:, :] = np.linalg.inv(D)

    return freq, Y


In [3]:
alpha=0.019790752586283235
beta= 0.00020062717328036815

In [4]:
mA = np.array([1,1,1,1])
kA = np.array([200,200,200])

In [5]:
np.save('Y_A', FRF_matrix(mA, kA, alpha, beta)[1])

In [6]:
np.save('freq', FRF_matrix(mA, kA, alpha, beta)[0])

In [7]:
mB = np.array([2,2,2,2])
kB = np.array([100,100,100])

In [8]:
np.save('Y_B', FRF_matrix(mB, kB,alpha, beta)[1])

In [9]:
mAB = np.zeros(5)
kAB = np.zeros(4)

In [10]:
mAB[:4] += mA
mAB[1:] += mB
mAB

array([1., 3., 3., 3., 2.])

In [11]:
kAB[:3] += kA
kAB[1:] += kB
kAB

array([200., 300., 300., 100.])

In [12]:
np.save('Y_AB', FRF_matrix(mAB, kAB,alpha, beta)[1])